In [ ]:
import torch
from sklearn import metrics
import multiprocessing as mp
from MGCL import MGCL
import os,csv,re, time
import pickle
import random
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from scipy import stats
from scipy.sparse import issparse
import scanpy as sc
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import cv2
import ST as ST
from IPython.display import Image

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

n_clusters = 7
dataset = '151673'

adata = sc.read("/home/dingsq/dsq/MGCL-project/Data/151673/sample_data.h5ad")
adata.var_names_make_unique()

/opt/anaconda3/envs/dsq_env/lib/python3.8/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
# define model
model = MGCL.MGCL(adata, device=device, alpha=6)

# train model
adata = model.train()

Multi-scale-new graph constructed!
Begin to train ST data...


100%|██████████| 600/600 [01:41<00:00,  5.94it/s]


Optimization finished for ST data!


In [3]:
radius = 50

tool = 'leiden' # mclust, leiden, and louvain

# clustering
from MGCL.utils import clustering

if tool == 'mclust':
   clustering(adata, n_clusters, radius=radius, method=tool, refinement=True) # For DLPFC dataset, we use optional refinement step.
elif tool in ['leiden', 'louvain']:
   clustering(adata, n_clusters, radius=radius, method=tool, start=0.1, end=1.1, increment=0.01, refinement=True)

Searching resolution...


/opt/anaconda3/envs/dsq_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


resolution=1.0899999999999996, cluster number=10
resolution=1.0799999999999996, cluster number=10
resolution=1.0699999999999996, cluster number=10
resolution=1.0599999999999996, cluster number=10
resolution=1.0499999999999996, cluster number=10
resolution=1.0399999999999996, cluster number=10
resolution=1.0299999999999996, cluster number=10
resolution=1.0199999999999996, cluster number=10
resolution=1.0099999999999996, cluster number=10
resolution=0.9999999999999996, cluster number=10
resolution=0.9899999999999995, cluster number=10
resolution=0.9799999999999995, cluster number=9
resolution=0.9699999999999995, cluster number=9
resolution=0.9599999999999995, cluster number=10
resolution=0.9499999999999995, cluster number=9
resolution=0.9399999999999996, cluster number=10
resolution=0.9299999999999996, cluster number=10
resolution=0.9199999999999996, cluster number=10
resolution=0.9099999999999996, cluster number=10
resolution=0.8999999999999996, cluster number=9
resolution=0.88999999999

In [4]:
import numpy as np
import anndata as ad
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, fowlkes_mallows_score, homogeneity_score, completeness_score, v_measure_score

label_file_path = "/home/dingsq/dsq/Data/151673/label_data.csv"
label_df = pd.read_csv(label_file_path, sep=',')  # 根据实际情况调整分隔符

# 2. 提取标签数据中的 cell_code 和 type 列
cell_codes_in_labels = label_df['cell_code'].tolist()  # 标签中的 cell_code
label_types = label_df['type'].str.strip().tolist()  # 标签中的 type

# 3. 初始化一个字典来存储 cell_code 到 type 的映射
label_dict = {cell_code: label_type for cell_code, label_type in zip(cell_codes_in_labels, label_types)}

# 4. 遍历 adata 中的每个样本，根据 cell_code 设置 ground_truth
#adata.obs['ground_truth'] = None  # 初始化 ground_truth 列

for cell_code in label_dict:
    if cell_code in adata.obs.index:
    # 3. 将 ground_truth 值改为 1
        adata.obs.at[cell_code, 'ground_truth'] = label_dict[cell_code]

adata = adata[~pd.isnull(adata.obs['ground_truth'])]
print(adata.shape)
# calculate metric ARI
ACC = metrics.accuracy_score(adata.obs['domain'], adata.obs['ground_truth'])
NMI = metrics.normalized_mutual_info_score(adata.obs['domain'], adata.obs['ground_truth'])
ARI = metrics.adjusted_rand_score(adata.obs['domain'], adata.obs['ground_truth'])
FMI = fowlkes_mallows_score(adata.obs['domain'], adata.obs['ground_truth'])
homogeneity = homogeneity_score(adata.obs['domain'], adata.obs['ground_truth'])
completeness = completeness_score(adata.obs['domain'], adata.obs['ground_truth'])
v_measure = v_measure_score(adata.obs['domain'], adata.obs['ground_truth'])
print('ARI:', ARI)
print('NMI:', NMI)
print('ARI:', ARI)
print('FMI:', FMI)
print('Homogeneity:', homogeneity)
print('Completeness:', completeness)
print('V_measure:', v_measure)

(3611, 33538)
ARI: 0.5276629246391944
NMI: 0.6679098174512333
ARI: 0.5276629246391944
FMI: 0.6122141166388868
Homogeneity: 0.6642825659819916
Completeness: 0.6715768989284447
V_measure: 0.6679098174512332


In [ ]:
counts=sc.read("/home/dingsq/dsq/MGCL-ST-project/Data/151673/sample_data.h5ad")
#Read in hitology image
img=cv2.imread("/home/dingsq/dsq/MGCL-ST-project/Data/151673/151673_full_image.tif")

In [ ]:
resize_factor=1000/np.min(img.shape[0:2])
resize_width=int(img.shape[1]*resize_factor)
resize_height=int(img.shape[0]*resize_factor)
counts.var.index=[i.upper() for i in counts.var.index]
counts.var_names_make_unique()
sc.pp.log1p(counts) # impute on log scale
if issparse(counts.X):counts.X=counts.X.A


In [ ]:
#Three different algorithms to detect contour, select the best one.Here we use cv2.

#-----------------1. Detect contour using cv2-----------------
cnt=ST.cv2_detect_contour(img, apertureSize=5,L2gradient = True)

binary=np.zeros((img.shape[0:2]), dtype=np.uint8)
cv2.drawContours(binary, [cnt], -1, (1), thickness=-1)
#Enlarged filter
cnt_enlarged = ST.scale_contour(cnt, 1.05)
binary_enlarged = np.zeros(img.shape[0:2])
cv2.drawContours(binary_enlarged, [cnt_enlarged], -1, (1), thickness=-1)
img_new = img.copy()
cv2.drawContours(img_new, [cnt], -1, (255), thickness=50)
img_new=cv2.resize(img_new, ((resize_width, resize_height)))


In [ ]:
res=50
# Note, if the numer of superpixels is too large and take too long, you can increase the res to 100
enhanced_exp_adata=ST.imputation(img=img, raw=counts, cnt=cnt, genes=counts.var.index.tolist(), shape="None", res=res, s=1, k=2, num_nbs=10)